In [19]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# confirm TensorFlow sees the GPU
from tensorflow.python.client import device_lib
assert 'GPU' in str(device_lib.list_local_devices())

# confirm Keras sees the GPU
from keras import backend
assert len(backend.tensorflow_backend._get_available_gpus()) > 0

#Uncomment to run on CPU
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [20]:
'''def read_images():
    training_images = []
    i = 0
    DIR = "E:\Galaxy Zoo\Data\images_training_rev1\images_training_rev1"
    for img in os.listdir(DIR):
        im = cv2.imread(os.path.join(DIR,img))
        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212
        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106
        training_images.append(im)
        i+=1
        if i==10000:
            break
        
    return training_images'''
    

'def read_images():\n    training_images = []\n    i = 0\n    DIR = "E:\\Galaxy Zoo\\Data\\images_training_rev1\\images_training_rev1"\n    for img in os.listdir(DIR):\n        im = cv2.imread(os.path.join(DIR,img))\n        im = im[106:106*3,106:106*3,:]          #cropping 424x424->212x212\n        im = cv2.resize(im,(106,106), interpolation=cv2.INTER_CUBIC)   #downsampling to half resolution 106x106\n        training_images.append(im)\n        i+=1\n        if i==10000:\n            break\n        \n    return training_images'

In [21]:
DIR = "E:\Galaxy Zoo\Data\images_training_rev1"

train_path = DIR + "\Train"
validate_path = DIR + "\Validate"

df = pd.read_csv("training_solutions_rev1/training_solutions_rev1.csv")

df.set_index('GalaxyID', inplace = True)

In [22]:
def img_processor(path):
    img = cv2.imread(path)
    img = img[106:106*3,106:106*3,:]
    img = cv2.resize(img,(106,106), interpolation = cv2.INTER_CUBIC)
    return img

def Batch_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_train = df.loc[ind].values
        X_train = np.array(im)
        X_train = X_train.reshape(1,106,106,3)
        y_train = y_train.reshape(1,37)
        yield(X_train,y_train)
        
def Validation_generator(DIR):
    for img in os.listdir(DIR):
        im = img_processor(os.path.join(DIR,img))
        ind = int(img[:-4])
        y_valid = df.loc[ind].values
        X_valid = np.array(im)
        X_valid = X_valid.reshape(1,106,106,3)
        y_valid = y_valid.reshape(1,37)
        yield(X_valid,y_valid)

In [23]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Conv2D,MaxPooling2D,ZeroPadding2D,Lambda
from keras.optimizers import SGD, RMSprop, Adam

In [24]:
def Conv(model,layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1))) 
        model.add(Conv2D((filters), kernel_size = (3,3), activation = 'relu'))
        
    model.add(MaxPooling2D(pool_size=(2,2), strides = (2,2)))
    
def FC_block(model,filters):
    model.add(Dense(filters, activation='relu'))
    model.add(Dropout(0.5))
    
def VGG16():
    model = Sequential()
    model.add(Lambda(lambda x : x, input_shape=(106,106,3)))

    Conv(model,2,64)
    Conv(model,2,128)
    Conv(model,3,256)
    Conv(model,3,512)
    Conv(model,3,512)
    
    model.add(Flatten())
    FC_block(model,4096)
    FC_block(model,4096)
    FC_block(model,1000)
    
    model.add(Dense(37))
    model.add(Activation("sigmoid"))

    return model

In [25]:

model = VGG16()

model.compile(optimizer = RMSprop(lr=1e-6), loss='mean_squared_error', metrics = ['accuracy'])


In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_3 (Lambda)            (None, 106, 106, 3)       0         
_________________________________________________________________
zero_padding2d_27 (ZeroPaddi (None, 108, 108, 3)       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 106, 106, 64)      1792      
_________________________________________________________________
zero_padding2d_28 (ZeroPaddi (None, 108, 108, 64)      0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 106, 106, 64)      36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 53, 53, 64)        0         
_________________________________________________________________
zero_padding2d_29 (ZeroPaddi (None, 55, 55, 64)        0         
__________

In [29]:
batch_size = 32

steps_per_batch = int(len(os.listdir(train_path))/batch_size)
val_steps_per_batch = int(len(os.listdir(validate_path))/batch_size)

In [30]:
import time
t1 = time.time()
model.fit_generator(Batch_generator(train_path),
                    samples_per_epoch=steps_per_batch,
                    epochs = 50,
                    validation_data=Validation_generator(validate_path),
                    nb_val_samples=val_steps_per_batch,
                    verbose = 1
                   )
t2 = time.time()
print(t1-t2)

Epoch 1/50
1924/1924 [==============================] - 142s 74ms/step - loss: 0.0430 - acc: 0.5312 - val_loss: 0.0302 - val_acc: 0.5825
Epoch 2/50
1924/1924 [==============================] - 143s 75ms/step - loss: 0.0390 - acc: 0.5177 - val_loss: 0.0285 - val_acc: 0.5908
Epoch 3/50
1924/1924 [==============================] - 143s 74ms/step - loss: 0.0368 - acc: 0.5281 - val_loss: 0.0280 - val_acc: 0.5678
Epoch 4/50
1924/1924 [==============================] - 145s 75ms/step - loss: 0.0354 - acc: 0.5494 - val_loss: 0.0265 - val_acc: 0.6033
Epoch 5/50
1924/1924 [==============================] - 147s 76ms/step - loss: 0.0344 - acc: 0.5421 - val_loss: 0.0272 - val_acc: 0.5992
Epoch 6/50
1924/1924 [==============================] - 147s 76ms/step - loss: 0.0330 - acc: 0.5582 - val_loss: 0.0268 - val_acc: 0.5992
Epoch 7/50
1924/1924 [==============================] - 148s 77ms/step - loss: 0.0318 - acc: 0.5551 - val_loss: 0.0244 - val_acc: 0.6033
Epoch 8/50
1924/1924 [===================

StopIteration: 

In [15]:
model.save("100819.model")